# Modelltraining Schilder Klassifizierung

## Bearbeiter
**TINF19-IT2**  
Patrick Küsters (9815596)  
Nick Kramer (3122448)  
**TINF19-IT1**  
Sören Holzenkamp (8528927)

## Imports
Lokal entwickelte Skripte und Module

In [1]:
import sys
from pathlib import Path
module_path = str(Path.cwd() / "src")
if module_path not in sys.path:
    sys.path.append(module_path)
from reader import Reader
from display_data import ImageDisplayer
from image_detection import predict_and_display_img
from evaluation import plot_confusion_matrix
import collage

Externe Bibliotheken

In [2]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AvgPool2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from skimage import exposure, transform
import matplotlib.pyplot as plt
import keras_tuner as kt
import tensorflow as tf
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np
import mimetypes
import pickle
import cv2
import os

In [3]:
file = open(r"./pickle/data_3.pickle", 'rb')
data = pickle.load(file)
file.close()
file = open(r"./pickle/labels_3.pickle", 'rb')
labels = pickle.load(file)
file.close()

In [4]:
input_shape = data[0].shape

In [5]:
targets = []
input_shape = data[0].shape
for k in labels:
    a = list(k.keys())
    a.sort(key=lambda x: (x[0], x[1])) ## wichtig! Erläuterung: nächste Zelle
    temp = [] #input vektor for model
    for e in a:
        x1, y1, x2, y2 = e #extract coordinates
        h, w = input_shape[:2]
        temp.append(x1/w) #scale with image shape
        temp.append(y1/h)
        temp.append(x2/w)
        temp.append(y2/h)
    targets.append(temp)
    
# convert the data and targets to NumPy arrays, scaling the input
# pixel intensities from the range [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")

In [6]:
split = train_test_split(data, targets, test_size=0.25, random_state=42)

(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]

split = train_test_split(testImages, testTargets, test_size=0.5, random_state=42)

(validationImages, testImages) = split[:2]
(validationTargets, testTargets) = split[2:4]

In [13]:
input_layer = Input(input_shape)

#create the base layers
base_layers = Conv2D(16, 3, padding='same', activation='relu', name='bl_2')(input_layer)
base_layers = MaxPooling2D(name='bl_3')(base_layers)
base_layers = Conv2D(32, 3, padding='same', activation='relu', name='bl_4')(base_layers)
base_layers = MaxPooling2D(name='bl_5')(base_layers)
base_layers = Conv2D(64, 3, padding='same', activation='relu', name='bl_6')(base_layers)
base_layers = MaxPooling2D(name='bl_7')(base_layers)
base_layers = Flatten(name='bl_8')(base_layers)

#create the localiser branch
locator_branch = Dense(128, activation='relu', name='bb_1')(base_layers)
locator_branch = Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = Dense(4, activation='sigmoid', name='bb_head')(locator_branch)

model = tf.keras.Model(input_layer, outputs=locator_branch)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 400, 680, 3)]     0         
                                                                 
 bl_2 (Conv2D)               (None, 400, 680, 16)      448       
                                                                 
 bl_3 (MaxPooling2D)         (None, 200, 340, 16)      0         
                                                                 
 bl_4 (Conv2D)               (None, 200, 340, 32)      4640      
                                                                 
 bl_5 (MaxPooling2D)         (None, 100, 170, 32)      0         
                                                                 
 bl_6 (Conv2D)               (None, 100, 170, 64)      18496     
                                                                 
 bl_7 (MaxPooling2D)         (None, 50, 85, 64)        0   

In [14]:
#loss = tf.keras.losses.MAE

In [27]:
import tensorflow.keras.backend as kb
def custom_loss(y_actual,y_pred):
    print(y_actual.shape)
    print(y_actual[0].shape)
    print(y_actual[0][0].shape)
    wu = y_actual[2] - y_actual[0]
    hu = y_actual[1] - y_actual[0]
    wv = y_pred[2] - y_pred[0]
    hv = y_pred[1] - y_pred[0]
    xu = y_actual[0]
    yu = y_actual[1]
    xv = y_pred[0]
    yv = y_pred[1]
    au = wu*hu
    av = wv*hv
    iw = kb.min(xu+0.5*wu, xv+0.5*wv)-kb.max(xu-0.5*wu, xv-0.5*wv)
    ih = kb.min(yu+0.5*hu, yv+0.5*hv)-kb.max(yu-0.5*hu, yv-0.5*hv)
    iq = kb.max(iw, 0)
    ih = kb.max(ih, 0)
    i = iw*ih
    u = au+av-i
    iou = i/u
    l = kb.log(iou+0.0001)
    return l
loss = custom_loss

In [28]:
model.compile(loss=loss, optimizer='Adam')

In [29]:
epoch_num = 10

In [30]:
training = model.fit(trainImages, trainTargets,
             validation_data=(testImages, testTargets),
             batch_size=10,
             epochs=epoch_num,
             shuffle=True,
             verbose=1)

2022-03-21 01:53:14.745160: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 24480000000 exceeds 10% of free system memory.


Epoch 1/10
(10, 4)
(4,)
()


TypeError: in user code:

    File "/home/soeren/anaconda3/envs/ml/lib/python3.10/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_9412/3945805092.py", line 16, in custom_loss  *
        iw = kb.min(xu+0.5*wu, xv+0.5*wv)-kb.max(xu-0.5*wu, xv-0.5*wv)
    File "/home/soeren/anaconda3/envs/ml/lib/python3.10/site-packages/keras/backend.py", line 2435, in min
        return tf.reduce_min(x, axis, keepdims)

    TypeError: Value passed to parameter 'reduction_indices' has DataType float32 not in list of allowed values: int32, int64


In [ ]:
#model.save(r"./models/models_regr_3signs_1", save_format="h5")

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epoch_num), training.history["loss"], label="train_loss")
plt.plot(np.arange(0, epoch_num), training.history["val_loss"], label="val_loss")
plt.title("Bounding Box Regression Loss on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")
#plt.show()
plt.savefig('./images/training3_15_1.png')